In [ ]:
#import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg') # Must be before importing matplotlib.pyplot or pylab!
import matplotlib.pyplot as plt

import numpy as np
import sys
sys.path.insert(0,'/local2/home/tong/caffe-master/python')
import caffe

import os
from matplotlib import rc
rc('mathtext', default='regular')

from sklearn.metrics import confusion_matrix

caffe.set_mode_gpu()
caffe.set_device(0)

solver = caffe.SGDSolver('/local2/home/tong/fashionRecommendation/models/fashionNet_8/fashion_solver_8.prototxt')

# Continue!!!
solver.net.copy_from('/local2/home/tong/fashionRecommendation/models/VGG_M_2048/VGG_CNN_M_2048.caffemodel')
# solver.net.copy_from('/local2/home/tong/fashionRecommendation/models/fashionNet_2/training_record/t2.1/fashion_params_2_124200.caffemodel')

# Continue!!!
solver.test_nets[0].copy_from('/local2/home/tong/fashionRecommendation/models/VGG_M_2048/VGG_CNN_M_2048.caffemodel')
# solver.test_nets[0].copy_from('/local2/home/tong/fashionRecommendation/models/fashionNet_2/training_record/t2.1/fashion_params_2_124200.caffemodel')

# for saving caffemodel
net = caffe.Net('/local2/home/tong/fashionRecommendation/models/fashionNet_8/fashion_deploy_8.prototxt', caffe.TEST)


In [ ]:
def test_avg(test_iter,i_train,val_batch_size):
    
    avg_accu = 0
    avg_loss = 0

    val_data_fp_temp0 = str(i_train)
    val_data_fp_temp1 = ''
    
    for i in range(0,test_iter):
        
        if (i%50==0 or (i==(test_iter-1))):
            print("{}/{}\n".format(i+1,test_iter))
            
        solver.test_nets[0].forward()
        
        avg_accu = avg_accu + solver.test_nets[0].blobs['accuracy'].data
        avg_loss = avg_loss + solver.test_nets[0].blobs['rank_Loss'].data

        for t in range(0,val_batch_size):
            val_data_fp_temp0 += ' '+solver.test_nets[0].blobs['like'].data[t]
            val_data_fp_temp1 += ' '+solver.test_nets[0].blobs['like_n'].data[t]
        
        val_data_fp_temp0 += val_data_fp_temp1
        
    return (avg_accu/test_iter), (avg_loss/test_iter), val_data_fp_temp0


In [ ]:
recordDir = '/local2/home/tong/fashionRecommendation/models/fashionNet_8/training_record/t8.1_train_val/'

test_interval = 3232 # 1/10 training epoch
test_iter = 400 # 1/10 validation epoch
visual_interval = 4 # every 4 iters
train_iters = 32320 # 1 training epoch

k = 0
start_iter = 0 # Continue!!!
end_iter = start_iter+train_iters*8 # t8.1(next i=0)[8_0.0001*(1,100)] Continue!!!

params = net.params.keys()

solver.net.forward()

train_batch_size = len(solver.net.blobs['top'].data)
val_batch_size = len(solver.test_nets[0].blobs['top'].data)

train_accu = (start_iter+1)*0 # Continue!!!
train_los = (start_iter+1)*0 # Continue!!!

train_avg_accu = 0
train_avg_loss = 0

train_cur_accu = 0
train_cur_loss = 0

test_accu = 0
test_loss = 0

for i in range (start_iter,end_iter+1):
#     if (i != start_iter): # Continue!!!
    # train_avg_accu/loss.txt
    train_accu = train_accu + solver.net.blobs['accuracy'].data
    train_los = train_los + solver.net.blobs['rank_loss'].data
    train_avg_accu = train_accu/(i+1)
    train_avg_loss = train_los/(i+1)
    train_avg_accu_fp = open(recordDir+'train_avg_accu.txt','a')
    train_avg_loss_fp = open(recordDir+'train_avg_loss.txt','a')
    train_avg_accu_fp.write(str(i)+' '+str(train_avg_accu)+'\r\n')
    train_avg_loss_fp.write(str(i)+' '+str(train_avg_loss)+'\r\n')
    train_avg_accu_fp.close()
    train_avg_loss_fp.close()
    if(i%visual_interval==0):
        print("\nIters done:{}, train_avg_accu={}, train_avg_loss={}.\n".format(i,train_avg_accu,train_avg_loss))
    # train_bat_accu/loss.txt
    train_cur_accu = solver.net.blobs['accuracy'].data
    train_cur_loss = solver.net.blobs['rank_loss'].data 
    train_bat_accu_fp = open(recordDir+'train_bat_accu.txt','a')
    train_bat_loss_fp = open(recordDir+'train_bat_loss.txt','a')
    train_bat_accu_fp.write(str(i)+' '+str(train_cur_accu)+'\r\n')
    train_bat_loss_fp.write(str(i)+' '+str(train_cur_loss)+'\r\n')
    train_bat_accu_fp.close()
    train_bat_loss_fp.close()
    if(i%visual_interval==0):
        print("                 train_bat_accu={}, train_bat_loss={}.\n".format(train_cur_accu,train_cur_loss))
    # train_data.txt
    train_data_fp = open(recordDir+'train_data.txt','a')
    train_data_fp_temp0 = str(i)
    train_data_fp_temp1 = ''
    for t in range(0,train_batch_size):
        train_data_fp_temp0 += ' '+solver.net.blobs['like'].data[t]
        train_data_fp_temp1 += ' '+solver.net.blobs['like_n'].data[t]
    train_data_fp_temp0 += train_data_fp_temp1
    train_data_fp.write(train_data_fp_temp0+'\r\n')
    train_data_fp.close()   
    # validation, save caffemodel, visualization, and stop criteria
    if (i%test_interval==0 or i==end_iter):
        # val_bat_accu/loss.txt
        test_accu,test_loss,\
        val_data_fp_temp0
        = \
        test_avg(test_iter,i,val_batch_size)
        val_bat_accu_fp = open(recordDir+'val_bat_accu.txt','a')
        val_bat_loss_fp = open(recordDir+'val_bat_loss.txt','a')
        val_bat_accu_fp.write(str(i)+str(test_accu)+'\r\n')
        val_bat_loss_fp.write(str(i)+str(test_loss)+'\r\n')
        val_bat_accu_fp.close()
        val_bat_loss_fp.close()
        # val_data.txt
        val_data_fp = open(recordDir+'val_data.txt','a')
        val_data_fp.write(val_data_fp_temp0+'\r\n')
        val_data_fp.close()
        # save caffemodel
        source_params = {pr: (solver.net.params[pr][0].data,solver.net.params[pr][1].data) for pr in params}
        target_params = {pr: (net.params[pr][0].data,net.params[pr][1].data) for pr in params}
        for pr in params:
            target_params[pr][0][...] = source_params[pr][0] #weights
            target_params[pr][1][...] = source_params[pr][1] #bias
        net.save(recordDir+'fashion_params_8_'+str(i)+'.caffemodel') 
        # avg_bat_record.png
        train_accu_whole = []
        train_loss_whole = []
        test_accu_whole = []
        test_loss_whole = []
        train_accu_whole = open(recordDir+'train_avg_accu.txt').readlines() 
        train_loss_whole = open(recordDir+'train_avg_loss.txt').readlines()
        test_accu_whole = open(recordDir+'val_bat_accu.txt').readlines()
        test_loss_whole = open(recordDir+'val_bat_loss.txt').readlines()
        train_iter_idx = []
        train_loss = []
        train_accuracy = []
        test_iter_idx = []
        test_loss = []
        test_accuracy = []
        for i in range(0, len(train_accu_whole)):
            if i % 300 == 0: # 1/100 training epoch
                train_iter_idx.append(int(train_accu_whole[i].strip('\r\n').split(' ')[0]))
                train_loss.append(float(train_loss_whole[i].strip('\r\n').split(' ')[1]))
                train_accuracy.append(float(train_accu_whole[i].strip('\r\n').split(' ')[1]))
        for i in range(0, len(test_accu_whole)):
            test_iter_idx.append(int(test_accu_whole[i].strip('\r\n').split(' ')[0]))
            test_loss.append(float(test_loss_whole[i].strip('\r\n').split(' ')[1]))
            test_accuracy.append(float(test_accu_whole[i].strip('\r\n').split(' ')[1]))
        fig = plt.figure()
        ax_left = fig.add_subplot(111)
        ax_left.plot(train_iter_idx, train_loss, '--rp', label = 'Avg_T_Loss')
        ax_left.plot(test_iter_idx, test_loss, '--gp', label = 'Avg_V_Loss')
        ax_right = ax_left.twinx()
        ax_right.plot(train_iter_idx, train_accuracy, '-bp', label = 'Avg_T_Accuracy')
        ax_right.plot(test_iter_idx, test_accuracy, '-yp', label = 'Avg_V_Accuracy')
        lines_left, labels_left = ax_left.get_legend_handles_labels()
        lines_right, labels_right = ax_right.get_legend_handles_labels()
        ax_right.legend(lines_left + lines_right, labels_left + labels_right, loc=0)
        ax_left.grid()
        ax_left.set_xlabel("Training Iterations Done(n)")
        ax_left.set_ylabel("Loss")
        ax_right.set_ylabel("Accuracy")
        ax_right.set_title("HVA@({:.3f},{}), LVL@({:.3f},{})".format(max(test_accuracy),test_iter_idx[test_accuracy.index(max(test_accuracy))],min(test_loss),test_iter_idx[test_loss.index(min(test_loss))]))
        plt.savefig(recordDir+'avg_bat_record.png', bbox_inches='tight')
        plt.close('all')
        # bat_bat_record.png
        train_accu_whole = []
        train_loss_whole = []
        train_accu_whole = open(recordDir+'train_bat_accu.txt').readlines() 
        train_loss_whole = open(recordDir+'train_bat_loss.txt').readlines()
        train_iter_idx = []
        train_loss = []
        train_accuracy = []
        for i in range(0, len(train_accu_whole)):
            if i % 300 == 0: # 1/100 training epoch
                train_iter_idx.append(int(train_accu_whole[i].strip('\r\n').split(' ')[0]))
                train_loss.append(float(train_loss_whole[i].strip('\r\n').split(' ')[1]))
                train_accuracy.append(float(train_accu_whole[i].strip('\r\n').split(' ')[1]))
        fig = plt.figure()
        ax_left = fig.add_subplot(111)
        ax_left.plot(train_iter_idx, train_loss, '--rp', label = 'Bat_T_Loss')
        ax_left.plot(test_iter_idx, test_loss, '--gp', label = 'Avg_V_Loss')
        ax_right = ax_left.twinx()
        ax_right.plot(train_iter_idx, train_accuracy, '-bp', label = 'Bat_T_Accuracy')
        ax_right.plot(test_iter_idx, test_accuracy, '-yp', label = 'Avg_V_Accuracy')
        lines_left, labels_left = ax_left.get_legend_handles_labels()
        lines_right, labels_right = ax_right.get_legend_handles_labels()
        ax_right.legend(lines_left + lines_right, labels_left + labels_right, loc=0)
        ax_left.grid()
        ax_left.set_xlabel("Training Iterations Done(n)")
        ax_left.set_ylabel("Loss")
        ax_right.set_ylabel("Accuracy")
        ax_right.set_title("HVA@({:.3f},{}), LVL@({:.3f},{})".format(max(test_accuracy),test_iter_idx[test_accuracy.index(max(test_accuracy))],min(test_loss),test_iter_idx[test_loss.index(min(test_loss))]))
        plt.savefig(recordDir+'bat_bat_record.png', bbox_inches='tight')
        plt.close('all')
        # stop criteria
        if test_accu > 0.93:
            k = k + 1
            if (k > 10):
                print '\n\nValidation accuracy: {} > 0.93 counted for 10 times\n\n'.format(test_accu)
                break
    # update parameters
    solver.step(1)
